<a href="https://colab.research.google.com/github/lblogan14/master_tensorflow_keras/blob/master/ch18_debug_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import numpy as np

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

TensorFlow:1.12.0


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist', one_hot=True)
x_train = mnist.train.images
x_test = mnist.test.images
y_train = mnist.train.labels
y_test = mnist.test.labels

# parameters
n_y = 10  # 0-9 digits
n_x = 784  # total pixels

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [0]:
def mlp(x, num_inputs, num_outputs,num_layers,num_neurons):
    w=[]
    b=[]
    for i in range(num_layers):
        # weights
        w.append(tf.Variable(tf.random_normal( 
                              [num_inputs if i==0 else num_neurons[i-1], 
                               num_neurons[i]]), 
                             name="w_{0:04d}".format(i) 
                            ) 
                ) 
        # biases
        b.append(tf.Variable(tf.random_normal( 
                              [num_neurons[i]]), 
                             name="b_{0:04d}".format(i) 
                            ) 
                )                   
    w.append(tf.Variable(tf.random_normal(
                          [num_neurons[num_layers-1] if num_layers > 0 else num_inputs,
                           num_outputs]),name="w_out"))
    b.append(tf.Variable(tf.random_normal([num_outputs]),name="b_out"))
    
    # x is input layer
    layer = x
    # add hidden layers
    for i in range(num_layers):
        layer = tf.nn.relu(tf.matmul(layer, w[i]) + b[i])
    # add output layer
    layer = tf.matmul(layer, w[num_layers]) + b[num_layers]
    
    return layer

#Fetching tensor values with tf.Session.run()
The values are
returned as a NumPy array and can be printed or logged with Python statements. This is
the simplest and easiest approach, with the biggest drawback being that the computation
graph executes all the dependent paths, starting from the fetched tensor, and if those paths
include the training operations, then it advances one step or one epoch.

#Printing tensor values with tf.Print()
For debugging purposes you can use `tf.Print()`.

    tf.Print(input_,
             data,
             message=None,
             first_n=None,
             summarize=None,
             name=None)

* `input_` is a tensor that gets returned from the function without anything being
done to it
* `data` is the list of tensors that get printed
* `message` is a string that gets printed as a prefix to the printed output
* `first_n` represents the number of steps to print the output; if this value is
negative then the value is always printed whenever the path is executed
* `summarize` represents the number of elements to print from the tensor; by
default, only three elements are printed

Let us modify the MNIST MLP model to add the print statement:

In [0]:
tf.reset_default_graph()

In [0]:
num_layers = 2
num_neurons = [16,32]
learning_rate = 0.01
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

# input images
x_p = tf.placeholder(dtype=tf.float32, name="x_p", shape=[None, n_x]) 
# target output
y_p = tf.placeholder(dtype=tf.float32, name="y_p", shape=[None, n_y]) 

In [7]:
model = mlp(x=x_p, 
            num_inputs=n_x, 
            num_outputs=n_y, 
            num_layers=num_layers, 
            num_neurons=num_neurons)
# tf.Print()
model = tf.Print(input_=model,
                 data=[tf.argmax(model,1)],
                 message='y_hat=',
                 summarize=10,
                 first_n=5
                )

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

  `from __future__ import print_function`



In [8]:
# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_p))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = optimizer.minimize(loss)

#predictions_check = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
#accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
with tf.Session() as tfs:
        tfs.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
                feed_dict={x_p: X_batch, y_p: Y_batch}
                _,batch_loss = tfs.run([optimizer,loss], 
                                       feed_dict = feed_dict
                                      )
                epoch_loss += batch_loss 
            average_loss = epoch_loss / n_batches
            print("epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))

epoch: 0000   loss = 6.536878
epoch: 0001   loss = 2.225183
epoch: 0002   loss = 1.925202
epoch: 0003   loss = 1.798884
epoch: 0004   loss = 1.710164
epoch: 0005   loss = 1.639174
epoch: 0006   loss = 1.579504
epoch: 0007   loss = 1.529937
epoch: 0008   loss = 1.487940
epoch: 0009   loss = 1.450179


#Asserting on conditions with tf.Assert()
The
`tf.Assert()`` function takes a condition, and if the condition is false, it then prints the lists
of given tensors and throws `tf.errors.InvalidArgumentError`.

    tf.Assert(condition,
              data,
              summarize=None,
              name=None)
              
To make sure that the `tf.Assert()` operation gets executed, we need
to add it to the dependencies. For example, let us define an assertion to check that
all the inputs are positive:

`assert_op = tf.Assert(tf.reduce_all(tf.greater_equal(x,0)),[x])`
The, add `assert_op` to the dependencies at the time of defining the model,

    with tf.control_dependencies([assert_op]):
        # x is input layer
        layer = x
        # add hidden layers
        for i in range(num_layers):
            layer = tf.nn.relu(tf.matmul(layer, w[i]) + b[i])
        # add output layer
        layer = tf.matmul(layer, w[num_layers]) + b[num_layers]

In [0]:
tf.reset_default_graph()

In [0]:
num_layers = 2
num_neurons = [16,32]
learning_rate = 0.01
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

# input images
x_p = tf.placeholder(dtype=tf.float32, name="x_p", shape=[None, n_x]) 
# target output
y_p = tf.placeholder(dtype=tf.float32, name="y_p", shape=[None, n_y]) 

model = mlp(x=x_p, 
            num_inputs=n_x, 
            num_outputs=n_y, 
            num_layers=num_layers, 
            num_neurons=num_neurons)

model = tf.Print(input_=model,
                 data=[tf.argmax(model,1)],
                 message='y_hat=',
                 summarize=10,
                 first_n=5
                )

In [13]:
# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_p))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = optimizer.minimize(loss)

#predictions_check = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
#accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

with tf.Session() as tfs:
        tfs.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
                if epoch > 5:
                    X_batch = np.copy(X_batch)
                    X_batch[0,0]=-2
                feed_dict={x_p: X_batch, y_p: Y_batch}
                _,batch_loss = tfs.run([optimizer,loss], 
                                       feed_dict = feed_dict
                                      )
                epoch_loss += batch_loss 
            average_loss = epoch_loss / n_batches
            print("epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))

epoch: 0000   loss = 6.260224
epoch: 0001   loss = 2.239868
epoch: 0002   loss = 2.086190
epoch: 0003   loss = 1.993967
epoch: 0004   loss = 1.927934
epoch: 0005   loss = 1.876514
epoch: 0006   loss = 1.835091
epoch: 0007   loss = 1.799004
epoch: 0008   loss = 1.767816
epoch: 0009   loss = 1.734336


Apart from the `tf.Assert()` function, which can take any valid conditional expression,
TensorFlow provides the following assertion operations that check for specific conditions
and have a simple syntax:
* assert_equal
* assert_greater
* assert_greater_equal
* assert_integer
* assert_less
* assert_less_equal
*assert_negative
* assert_none_equal
* assert_non_negative
* assert_non_positive
* assert_positive
* assert_proper_iterable
* assert_rank
* assert_rank_at_least
* assert_rank_in
* assert_same_float_dtype
* assert_scalar
* assert_type
* assert_variables_initialized

#Debugging with the TensorFlow debugger (tfdbg)
To use a debugger,
1. Set the breakpoints in the code at locations where you want to break and inspect
the variables
2. Run the code in debug mode
3. When the code breaks at a breakpoint, inspect it and then move on to next step

In [0]:
tf.reset_default_graph()

In [0]:
num_layers = 2
num_neurons = [16,32]
learning_rate = 0.01
n_epochs = 10
batch_size = 100
n_batches = int(mnist.train.num_examples/batch_size)

# input images
x_p = tf.placeholder(dtype=tf.float32, name="x_p", shape=[None, n_x])
# target output
y_p = tf.placeholder(dtype=tf.float32, name="y_p", shape=[None, n_y])

model = mlp(x=x_p,
            num_inputs=n_x,
            num_outputs=n_y,
            num_layers=num_layers,
            num_neurons=num_neurons)

model = tf.Print(input_=model,
                 data=[tf.argmax(model,1)],
                 message='y_hat=',
                 summarize=10,
                 first_n=5
                )

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_p))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = optimizer.minimize(loss)

#predictions_check = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
#accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))
from tensorflow.python import debug as tfd

with tfd.LocalCLIDebugWrapperSession(tf.Session()) as tfs:
        tfs.run(tf.global_variables_initializer())
        tfs.add_tensor_filter('has_inf_or_nan_filter', tfd.has_inf_or_nan)
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = mnist.train.next_batch(batch_size)
                if epoch > 0:
                    X_batch = np.copy(X_batch)
                    X_batch[0,0]=np.inf
                feed_dict={x_p: X_batch, y_p: Y_batch}
                _,batch_loss = tfs.run([optimizer,loss],
                                       feed_dict = feed_dict
                                      )
                epoch_loss += batch_loss
            average_loss = epoch_loss / n_batches
            print("epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))